<a href="https://colab.research.google.com/github/NoCodeProgram/deepLearning/blob/main/nn/classification_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NoCodeProgram/deepLearning.git


Cloning into 'deepLearning'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 56 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (56/56), 1.85 MiB | 4.47 MiB/s, done.


In [ ]:
import torch
import pandas as pd

# Load the DataFrame from a CSV file
df = pd.read_csv('./deepLearning/nn/classify5k.csv')

# Convert the DataFrame to a numpy array
data = df[['x', 'y']].values
labels = df['label'].values.reshape(-1, 1)

# Print the shapes of the data and labels
print(f'Data shape:{data.shape}')
print(f'Labels shape:{labels.shape}')


In [ ]:
import matplotlib.pyplot as plt

# Plot the points
plt.figure(figsize=(6, 6))
plt.scatter(data[:,0], data[:,1], c=labels, cmap='viridis', s=1)
plt.title('Sample Data with Noisy Labels')
plt.xlabel('x')
plt.ylabel('y')
plt.show()


In [ ]:
# Convert the numpy arrays to PyTorch tensors
data_torch = torch.tensor(data, dtype=torch.float32)
labels_torch = torch.tensor(labels, dtype=torch.float32)
data_length = len(data_torch)
split_length = int(0.8*data_length)

train_data = data_torch[:split_length]
train_labels = labels_torch[:split_length]
val_data = data_torch[split_length:]
val_labels = labels_torch[split_length:]

print(train_data.shape, train_labels.shape)

In [ ]:
g = torch.Generator().manual_seed(42)

def get_batch(data, labels, batch_size=256):
    # Generate random indices
    indices = torch.randint(0, len(data), size=(batch_size,), generator= g)

    # Select the data and labels at these indices
    data_batch = data[indices]
    labels_batch = labels[indices]

    return data_batch, labels_batch

data_batch, labels_batch = get_batch(train_data,train_labels)
print(data_batch.shape, labels_batch.shape)

In [ ]:
import torch.nn.functional as F

input_size = 2
hidden_size = 4
output_size = 1

g.manual_seed(42)
W1 = torch.randn((input_size, hidden_size), generator=g)
b1 = torch.randn(hidden_size, generator=g)
W2 = torch.randn((hidden_size, output_size) , generator=g)
b2 = torch.randn(output_size, generator=g)
params = [W1,b1,W2,b2]
for p in params:
    p.requires_grad = True

for steps in range(200000):
    data_batch, labels_batch = get_batch(train_data,train_labels, batch_size=256)
    tmp = data_batch@W1 + b1
    tmp = F.relu(tmp)
    output = tmp@W2 + b2
    prob = torch.sigmoid(output)
    loss = -1 * (labels_batch * torch.log(prob) + (1 - labels_batch) * torch.log(1 - prob)).mean()

    loss.backward()
    with torch.no_grad():
        W1 -= 0.01 * W1.grad
        b1 -= 0.01 * b1.grad
        W2 -= 0.01 * W2.grad
        b2 -= 0.01 * b2.grad
        W1.grad.zero_()
        b1.grad.zero_()
        W2.grad.zero_()
        b2.grad.zero_()

    if steps % 1000 == 0:
        tmp = val_data@W1 + b1
        tmp = F.relu(tmp)
        output = tmp@W2 + b2
        prob = torch.sigmoid(output)
        val_loss = -1 * (val_labels * torch.log(prob) + (1 - val_labels) * torch.log(1 - prob)).mean()
        print(f"{steps},val_loss: {val_loss},train_loss: {loss}" )



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate 10000 new points
n_points_test = 100000
points_test = np.random.uniform(-1, 1, (n_points_test, 2))
points_test_torch = torch.from_numpy(points_test).float()

# Run the points through the network
with torch.no_grad():  # We don't need gradients for this part
    tmp = points_test_torch@W1 + b1
    tmp = F.relu(tmp)
    output_test = tmp@W2 + b2
    prob = torch.sigmoid(output_test)
pred = (0.5 < prob).numpy()

# Plot the points, colored by their predicted class
plt.figure(figsize=(6, 6))
plt.scatter(points_test[:, 0], points_test[:, 1], c=pred, cmap='viridis', s=1)
plt.title('Predicted Classes for Test Points')
plt.xlabel('x')
plt.ylabel('y')
plt.show()
